#  Building a Postgres database for crime reports in Boston


The goal of this guided project is to setup a database of Boston crime data from scratch.

We will create two user groups:

* `readonly`: users in this group will have permission to read data only.

* `readwrite`: users in this group will have permissions to read and alter data but not to delete tables

## Creating the database and the schema

In [1]:
import psycopg2
import csv
import io
from datetime import datetime

conn = psycopg2.connect(dbname='dq', user='dq')
# set autocommit to True bacause this is required for creating databases
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE DATABASE crime_db;")
conn.close()


In [2]:
conn = psycopg2.connect(dbname='crime_db', user='dq')
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes;")

## Obtaining the Column Names and Sample Data

In [3]:
with open('boston.csv') as file:
    read_file = csv.reader(file)
    col_headers = next(read_file)
    first_row = next(read_file)
    
print(col_headers, '\n')
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long'] 

['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


## Creating an Auxiliary Function

create a function — get_col_set() — that, given the name of a CSV file and a column index (starting at 0), computes a Python set with all distinct values contained in that column.

In [4]:
def get_col_set(csv_filename, col_index):
    values = set()
    with open(csv_filename, 'r') as file:
        read_file = csv.reader(file)
        header = next(read_file)
        for row in read_file:
            values.add(row[col_index])
    return values

In [5]:
for i in range(len(col_headers)):
    values = get_col_set('boston.csv', i)
    print(col_headers[i], len(values), sep='\t')
    

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


## Finding the Maximum Length of Columns containing textual data

Use the get_col_set() function to get the distinct values of that column into a variable named col_values.

Iterate over the elements of col_values, and find the longest one

In [6]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [7]:
descriptions = get_col_set('boston.csv', 2) # description is at index 2
max_len = 0
for description in descriptions:
    max_len = max(max_len, len(description))
print(max_len)

58


## Creating the Table


We have created an enumerated datatype named `weekday` for the day_of_the_week since there there are only seven possible values.

For the incident_number, we have decided to user the type INTEGER and set it as the primary key. The same datatype was also used to represent the offense_code.

Since the description has at most 58 characters, we decided to use the datatype VARCHAR(100) for representing it. This leaves some margin while not being so big that we will waste a lot of memory.

The date was represented as the DATE datatype. Finally, for the latitude and longitude, we used DECIMAL datatypes.

In [8]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [9]:
# Create the enumerated datatype for representing the weekday.
cur.execute("""
    CREATE TYPE weekday AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');
""")
# Create the table.
cur.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(100),
        date DATE,
        day_of_the_week weekday,
        lat decimal,
        long decimal
    );
""")

## Loading the Data Into Table

Load the data from the `"boston.csv"` file into the `"crimes.boston_crimes"` table

In [10]:
with open('boston.csv') as file:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", file)


In [11]:
cur.execute("SELECT * FROM crimes.boston_crimes;")
result = cur.fetchmany(5)
for row in result:
    print(row)
    
print(len(cur.fetchall()), '\n')

(1, 619, 'LARCENY ALL OTHERS', datetime.date(2018, 9, 2), 'Sunday', Decimal('42.35779134'), Decimal('-71.13937053'))
(2, 1402, 'VANDALISM', datetime.date(2018, 8, 21), 'Tuesday', Decimal('42.30682138'), Decimal('-71.06030035'))
(3, 3410, 'TOWED MOTOR VEHICLE', datetime.date(2018, 9, 3), 'Monday', Decimal('42.34658879'), Decimal('-71.07242943'))
(4, 3114, 'INVESTIGATE PROPERTY', datetime.date(2018, 9, 3), 'Monday', Decimal('42.33418175'), Decimal('-71.07866441'))
(5, 3114, 'INVESTIGATE PROPERTY', datetime.date(2018, 9, 3), 'Monday', Decimal('42.27536542'), Decimal('-71.09036101'))
298324 



## Revoke All Public Privileges

* Revoke all the privileges of the public group on the public schema.
* Revoke all privileges of public on the crime_db database.

In [12]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

## Creating User Groups



In [16]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("CREATE GROUP readwrite;")

cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")


ProgrammingError: role "readonly" already exists


## Granting Specific Privileges to Each Group

In [17]:
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")

cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

## Creating Users and Assigning to Groups

In [18]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")

cur.execute("GRANT readonly TO data_analyst;")
cur.execute("GRANT readwrite TO data_scientist;")

## Testing

test the setup using SQL that everything is configured as expected when finished setting up the database. We can query Postgres internal tables to see this information.

In [19]:
# Close the old connection to test with a brand new connection.
conn.close()

conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()
# Check users and groups.
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
for user in cur:
    print(user)
print()


# check privileges
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
for user in cur:
    print(user)
conn.close()

('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)

('readonly', 'SELECT')
('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')
